In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langsmith import Client

load_dotenv()

## Basics

In [4]:
model = ChatOpenAI(model="gpt-4.1-mini", seed=42, temperature=0)

messages = [
    SystemMessage("You're Jose. You're a helpful assistant that replies in haikus."),
    HumanMessage("What is the color of the sky?"),
]

response = model.invoke(messages)

## Prompt templates

### Prompt files (Using Jinja2)

In [ ]:
from langchain.prompts import PromptTemplate


def load_prompt(prompt_filename, partial_variables=None):
    partial_variables = partial_variables or {}
    with open(prompt_filename, "r") as f:
        file_content = f.read()
        return PromptTemplate.from_template(
            file_content, template_format="jinja2", partial_variables=partial_variables
        ).format()


questions = ["What is the color of the sky?", "What is the color of the grass?"]

messages = [
    SystemMessage(load_prompt("assets/system_prompt.jinja2")),
    HumanMessage(
        load_prompt(
            "assets/user_prompt.jinja2",
            # IMPORTANT: must sanitize the input to avoid Jinja2 injection
            partial_variables={"questions": questions},
        )
    ),
]

response = model.invoke(messages)

### Prompt catalog (Using LangSmith)

In [48]:
client = Client()
prompts = client.pull_prompt("workshop_system_prompt")
messages = prompts.format_messages(question="What is the color of the sky?")

response = model.invoke(messages)

## Streaming

In [ ]:
chunks = []
for chunk in model.stream(messages):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

## Multimodality

In [ ]:
import base64


def image_to_base64_string(image_path):
    with open(image_path, "rb") as image_file:
        s = base64.b64encode(image_file.read())
        return s.decode("utf-8")


messages = [
    SystemMessage(
        "You're Jose. You're a helpful assistant who always replies in haikus."
    ),
    HumanMessage(
        content=[
            {"type": "text", "text": "Describe this image:"},
            {
                "type": "image",
                "source_type": "base64",
                "data": image_to_base64_string("assets/dog_image.png"),
                "mime_type": "image/png",
            },
        ],
    ),
]
response = model.invoke(messages)

# Exercise

Create a new set of prompts in the prompt catalog. In it the user should be able to specify the style of the response in addition to the question.

In [ ]:
client = Client()
prompts = client.pull_prompt("workshop_homework")
messages = prompts.format_messages(question="What is the color of the sky?", style="formal")

model.invoke(messages)